# Data preprocessing and lemmatisation

In [ ]:
! pip install polyglot
! pip install pyicu
! pip install Morfessor       
! pip install pycld2   
! polyglot download LANG:pl

     |████████████████████████████████| 126 kB 11.8 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=c5a3e563196ba0d36a2f809adbd65c0275b386141f042f916416fcf68c2dfc40
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot
     |████████████████████████████████| 299 kB 9.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyicu: filename=PyICU-2.8-cp37-cp37m-linux_x86_64.whl size=1376696 sha256=506d812fc8600232bac85da01fb8d0c55fb1c6a91e53979a4c7bc739db8f7234
  Stored in directory: /root/.cache/pip/wheels/14/bd/45/aeddc643bd0637c14fa27bffaee5b411cdc323f8bec76ad15e
Successfully built pyicu
     |████████████████████████████████| 41.4 MB 1.7 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834267 sha256=2e71b53c6bc761fae20

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from polyglot.text import Text
from polyglot.detect import Detector
import string
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
def drop_title_and_url(df):
    return df.drop(columns=['Title', 'Url'])

def drop_empty(df):
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def drop_non_polish(df):
    for index, row in df.iterrows():
        text = row['Text']
        detector = Detector(text, quiet=True)
        if not (detector.language.name == 'Polish' and 
                detector.language.confidence >= 70):
            df.drop([index], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def drop_unidentified(df):
    return df[df['Verdict'] != 'unidentified']

def drop_twitter(df):
    return df[df['Text'].str.contains('Nowy na Twitterze')==False]

In [ ]:
# Load fake news dataset
df_fake = pd.read_excel('drive/MyDrive/Fake News Detection/data/fakehunter_dataset.xlsx')

# Drop irrelevant columns 
df_fake = drop_title_and_url(df_fake)

# Drop rows containing NaNs
df_fake = drop_empty(df_fake)

# Drop news with "unidentified" verdict
df_fake = drop_unidentified(df_fake)

# Drop non-polish news
df_fake = drop_non_polish(df_fake)

# Drop noisy twitter news
df_fake = drop_twitter(df_fake)

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [ ]:
# Load reliable news dataset
df_real = pd.read_excel('drive/MyDrive/Fake News Detection/data/termedia_dataset.xlsx')

# Drop irrelevant columns
df_real = drop_title_and_url(df_real)

# Drop rows containing NaNs
df_real = drop_empty(df_real)

In [ ]:
# Create complete dataset (containing both fake and real news) and reset index
df = pd.concat([df_fake, df_real])
df.reset_index(drop=True, inplace=True)

# Transform "Verdict" column to boolean 
df['Verdict'].replace('false', 0, inplace=True)
df['Verdict'].replace('true', 1, inplace=True)
df['Verdict'] = df['Verdict'].astype(bool)

In [ ]:
! pip install morfeusz2

     |████████████████████████████████| 8.5 MB 9.0 MB/s 


In [ ]:
# Imports
import re
import string
import morfeusz2

In [ ]:
# def strip_punctuation(text):
#     return text.translate(str.maketrans('', '', string.punctuation))

def strip_non_alphanumeric(text):
    reg = re.compile('[^a-zA-ZĄąĆćĘęŁłŃńÓóŚśŹźŻż]')
    return reg.sub(' ', text)

def replace_whitespace(text):
    reg = re.compile('\s+')
    return reg.sub(' ', text)

def delete_escape_chars(text):
    return text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\r', ' ')

def delete_stop_words(text):
    stop_words_txt = open('drive/MyDrive/Fake News Detection/data/polish.stopwords.txt')
    stop_words = stop_words_txt.read().split('\n')
    stop_words_txt.close()
    return [word for word in text if word not in stop_words]

def preprocess(df):
    # df['Text'] = df['Text'].apply(strip_punctuation)
    df['Text'] = df['Text'].apply(strip_non_alphanumeric)
    df['Text'] = df['Text'].apply(replace_whitespace)
    df['Text'] = df['Text'].apply(delete_escape_chars)
    df['Text'] = df['Text'].apply(str.lower)
    df['Text'] = df['Text'].apply(str.split)
    df['Text'] = df['Text'].apply(delete_stop_words)
    return df

def lemmatise(df):
    morf = morfeusz2.Morfeusz()
    for index, row in df.iterrows():
        text = row['Text']
        lemm_words = []
        for word in text:
            _, _, interpretation = morf.analyse(word)[0]
            lem_word = interpretation[1]
            lem_word_stripped = lem_word.split(':', 1)[0].lower()
            lemm_words.append(lem_word_stripped)
        df.loc[index, 'Text'] = ' '.join(lemm_words)
    return df

In [ ]:
df = preprocess(df)
df = lemmatise(df)

In [ ]:
# Drop rows not containing text and reset index
df = df[df['Text'].astype(bool)]
df.reset_index(drop=True, inplace=True)

# Drop rows with text shorter than 30 chars
df = df[df['Text'].apply(len) >= 30]

# Display number of fake and real news
print(f'Number of fake news: \t' + str(len(df[df['Verdict'] == False])))
print(f'Number of real news: \t' + str(len(df[df['Verdict'] == True])))
print(f'Total number of news: \t' + str(len(df)))

# Export complete dataset
df.to_excel('complete_dataset.xlsx', encoding='utf-8', index=False)

Number of fake news: 	753
Number of real news: 	1487
Total number of news: 	2240
